In [ ]:
library(drc)
library(bmd)
library(dplyr)
library(ggplot2)

In [ ]:
m <- read.csv("/app/tests/res/PE2_germination_data.csv")
head(m)
tail(m)

In [ ]:
#calculating cumulative observations
m_cumu <- m %>%                                   #we create a new dataframe with an extra column for cumulative observations
  #group_by(experiment, rep, treatment) %>%        #cumulative observations are grouped by factors (here we have three factors, this can be adjusted for more or less factors)
    group_by(treatment) %>%        #cumulative observations are grouped by factors (here we have three factors, this can be adjusted for more or less factors)

  mutate(cumulative_observations = cumsum(obs))   #cumulative observations are stored in a new column

head(m_cumu)
tail(m_cumu)

In [ ]:
ggplot(m_cumu, aes(x = start, y = cumulative_observations)) +
  geom_point()

In [ ]:
#plot cumulative observations, grouped by treatment
ggplot(m_cumu, aes(x = start, y = cumulative_observations, color = as.factor(treatment))) +
  geom_point()+
  ggtitle("Cumulative observations per treatment")

In [ ]:
?drm
getMeanFunctions()

In [ ]:
model_LL<-drm(obs~start+end, # The formula (in terms of the variables in the data)
               type="event", # the type of the data
               fct=LL.2(), # the function of the model 
               data= m, # specifying the name of the data set
               curveid = treatment, # the curve identifier (grouping variable)
               separate = TRUE) # to fit every treatment independently

In [ ]:
model_W1<-drm(obs~start+end,type="event", fct=W1.2(),data= m, curveid = treatment, separate = TRUE)
model_W2<-drm(obs~start+end,type="event", fct=W2.2(),data= m, curveid = treatment, separate = TRUE)

In [ ]:
?AIC()

In [ ]:
aic_df_LL <- data.frame(
  Treatment = names(lapply(model_LL$objList, function(x) AIC(x))),
  AIC_LL3 = unlist(lapply(model_LL$objList, function(x) AIC(x)))
)

aic_df_W1 <- data.frame(
  Treatment = names(lapply(model_W1$objList, function(x) AIC(x))),
  AIC_W1 = unlist(lapply(model_W1$objList, function(x) AIC(x)))
)

aic_df_W2 <- data.frame(
  Treatment = names(lapply(model_W2$objList, function(x) AIC(x))),
  AIC_W2 = unlist(lapply(model_W2$objList, function(x) AIC(x)))
)

In [ ]:
AIC_df <- data.frame(
  Treatment = names(lapply(model_LL$objList, function(x) AIC(x))),
  AIC_LL = unlist(lapply(model_LL$objList, function(x) AIC(x))),
  AIC_W1 = unlist(lapply(model_W1$objList, function(x) AIC(x))),
  AIC_W2 = unlist(lapply(model_W2$objList, function(x) AIC(x)))
)
AIC_df$MinAIC <- 
  apply(AIC_df[, 2:4], 1, function(x) {colnames(AIC_df)[2:4][which.min(x)]}) #we create a column that displays the name of the model with the lowest AIC

rownames(AIC_df) <- NULL

print(AIC_df)

In [ ]:
plot(model_LL, log="", ylim=c(0,1), xlim=c(0,12), col=TRUE,lwd ="2",
     legendPos=c(2, 1.0), legendText = c("21 °C","15 °C", "5 °C","2.5 °C"),
     xlab="Days from experimental start", ylab="Fraction of germinated seeds",
     main= "Soy beans germinated at five temperatures")

In [ ]:
qplotDrc(model_LL,
         col=TRUE,
     xlab="Days from experimental start", ylab="Fraction of germinated seeds") +
    ggtitle("Soy beans germinated at five temperatures")+
    labs(col = "Temperature")

In [ ]:
model_LL$objList                  #parameters for each treatment
lapply(model_LL$objList, confint) #confidence interval for each parameter for each treatment